In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# pyOptSparseDriver

pyOptSparseDriver wraps the optimizer package pyOptSparse, which provides a common interface for 11 optimizers, some of which are included in the package (e.g., SLSQP and NSGA2), and some of which are commercial products that must be obtained from their respective authors (e.g. SNOPT). The pyOptSparse package is based on pyOpt, but adds support for sparse specification of constraint Jacobians. Most of the sparsity features are only applicable when using the SNOPT optimizer.

```{note}
The pyOptSparse package does not come included with the OpenMDAO installation. It is a separate optional package that can be obtained from [mdolab](https://github.com/mdolab/pyoptsparse).
```

In this simple example, we use the SLSQP optimizer to minimize the objective of SellarDerivativesGrouped.

In [2]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src18", get_code("openmdao.test_suite.components.sellar.SellarDerivativesGrouped"), display=False)

:::{Admonition} `SellarDerivativesGrouped` class definition 
:class: dropdown

{glue:}`code_src18`
:::

We use a utility class to create the Sellar problem with it's design variables, objective and constraints:

In [3]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_sellar_problem", get_code("openmdao.test_suite.components.sellar.SellarProblem"), display=False)

:::{Admonition} `SellarProblem` class definition 
:class: dropdown

{glue:}`code_sellar_problem`
:::

In [4]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem, SellarDerivativesGrouped

prob = SellarProblem(SellarDerivativesGrouped)
prob.setup(check=False, mode='rev')

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()
prob.model.mda.nonlinear_solver = om.NonlinearBlockGS()
prob.model.mda.linear_solver = om.ScipyKrylov()

prob.driver = om.pyOptSparseDriver(optimizer='SLSQP')

prob.run_driver()

RuntimeError: pyOptSparseDriver is not available, pyOptsparse is not installed.

In [ ]:
print(prob.get_val('z', indices=0))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob.get_val('z', indices=0), 1.9776, 1e-3)

## pyOptSparseDriver Options

In [ ]:
om.show_options_table("openmdao.drivers.pyoptsparse_driver.pyOptSparseDriver")

## pyOptSparseDriver Constructor

The call signature for the *pyOptSparseDriver* constructor is:

```{eval-rst}
    .. automethod:: openmdao.drivers.pyoptsparse_driver.pyOptSparseDriver.__init__
       :noindex:
```    

## Using pyOptSparseDriver

pyOptSparseDriver has a small number of unified options that can be specified as keyword arguments when it is instantiated or by using the “options” dictionary. We have already shown how to set the `optimizer` option. Next we see how the `print_results` option can be used to turn on or off the echoing of the results when the optimization finishes. The default is True, but here, we turn it off.

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem, SellarDerivativesGrouped

prob = SellarProblem(SellarDerivativesGrouped)
prob.setup(check=False, mode='rev')

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()
prob.model.mda.nonlinear_solver = om.NonlinearBlockGS()
prob.model.mda.linear_solver = om.ScipyKrylov()

prob.driver = om.pyOptSparseDriver(optimizer='SLSQP')

prob.driver.options['print_results'] = False

prob.run_driver()

print(prob.get_val('z', indices=0))

In [ ]:
assert_near_equal(prob.get_val('z', indices=0), 1.9776, 1e-3)

Every optimizer also has its own specialized settings that allow you to fine-tune the algorithm that it uses. You can access these within the `opt_setting` dictionary. These options are different for each optimizer, so to find out what they are, you need to read your optimizer's documentation. We present a few common ones here.


## SLSQP-Specific Settings

Here, we set a convergence tolerance for SLSQP:

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem, SellarDerivativesGrouped

prob = SellarProblem(SellarDerivativesGrouped)
prob.setup(check=False, mode='rev')

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()
prob.model.mda.nonlinear_solver = om.NonlinearBlockGS()
prob.model.mda.linear_solver = om.ScipyKrylov()

prob.driver = om.pyOptSparseDriver(optimizer='SLSQP')

prob.driver.opt_settings['ACC'] = 1e-9

prob.setup(check=False, mode='rev')
prob.run_driver()

In [ ]:
print(prob.get_val('z', indices=0))

In [ ]:
assert_near_equal(prob.get_val('z', indices=0), 1.9776, 1e-3)

Similarly, we can set an iteration limit. Here, we set it to just a few iterations, and don't quite reach the optimum.


In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem, SellarDerivativesGrouped

prob = SellarProblem(SellarDerivativesGrouped)
prob.setup(check=False, mode='rev')

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()
prob.model.mda.nonlinear_solver = om.NonlinearBlockGS()
prob.model.mda.linear_solver = om.ScipyKrylov()

prob.driver = om.pyOptSparseDriver(optimizer='SLSQP')

prob.driver.opt_settings['MAXIT'] = 3

prob.run_driver()

In [ ]:
print(prob.get_val('z', indices=0))

In [ ]:
assert_near_equal(prob.get_val('z', indices=0), 1.98337708, 1e-3)

## SNOPT-Specific Settings

SNOPT has many customizable settings. Here we show two common ones.

Setting the convergence tolerance:

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem, SellarDerivativesGrouped

prob = SellarProblem(SellarDerivativesGrouped)
prob.setup(check=False, mode='rev')

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()
prob.model.mda.nonlinear_solver = om.NonlinearBlockGS()
prob.model.mda.linear_solver = om.ScipyKrylov()

prob.driver = om.pyOptSparseDriver(optimizer='SNOPT')

prob.driver.opt_settings['Major feasibility tolerance'] = 1e-9

prob.run_driver()

In [ ]:
print(prob.get_val('z', indices=0))

In [ ]:
assert_near_equal(prob.get_val('z', indices=0), 1.9776, 1e-3)

Setting a limit on the number of major iterations. Here, we set it to just a few iterations, and don't quite reach the optimum.

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem, SellarDerivativesGrouped

prob = SellarProblem(SellarDerivativesGrouped)
prob.setup(check=False, mode='rev')

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()
prob.model.mda.nonlinear_solver = om.NonlinearBlockGS()
prob.model.mda.linear_solver = om.ScipyKrylov()

prob.driver = om.pyOptSparseDriver(optimizer='SNOPT')

prob.driver.opt_settings['Major iterations limit'] = 5

prob.run_driver()

In [ ]:
print(prob.get_val('z', indices=0))

In [ ]:
assert_near_equal(prob.get_val('z', indices=0), 1.9780247, 2e-3)

If you have pyoptsparse 1.1 or greater, then you can send a signal such as SIGUSR1 to a running SNOPT optimization to tell it to terminate cleanly. This is useful if an optimization has gotten close enough to an optimum.  How to do this is dependent on your operating system in all cases, on your mpi implementation if you are running mpi, and on your queuing software if you are on a supercomputing cluster. Here is a simple example for unix and mpi.

``` bash
    ktmoore1$ ps -ef |grep sig
      502 17955   951   0  4:05PM ttys000    0:00.02 mpirun -n 2 python sig_demo.py
      502 17956 17955   0  4:05PM ttys000    0:00.03 python sig_demo.py
      502 17957 17955   0  4:05PM ttys000    0:00.03 python sig_demo.py
      502 17959 17312   0  4:05PM ttys001    0:00.00 grep sig

    ktmoore1$ kill -SIGUSR1 17955
```

You can enable this feature by setting the "user_terminate_signal" option and giving it a signal (imported from the
signal library in Python).  By default, user_terminate_signal is None, which disables the feature.
Here, we set the signal to SIGUSR1:

In [ ]:
import openmdao.api as om
import signal

prob = om.Problem()

prob.driver = om.pyOptSparseDriver()
prob.driver.options['optimizer'] = "SNOPT"
prob.driver.options['user_terminate_signal'] = signal.SIGUSR1

You can learn more about the available options in the [SNOPT_Manual](http://www.sbsi-sol-optimize.com/manuals/SNOPT%20Manual.pdf).